In [1]:
import numpy as np

In [2]:
sizep = 225 #cols
gs = 4
pangl = 3
total_spectra = gs * pangl #rows

In [3]:
n = np.linspace(0.6, 0.5, sizep ) #(sizep,)
favn = np.repeat(n[np.newaxis,:], total_spectra, axis=0) #This is n repeated totalspectra, sizep

# Magic K
k = np.linspace(0.01,0.05, sizep) #(sizep,)
favk = np.repeat(k[np.newaxis,:], total_spectra, axis=0) #This is n repeated totalspectra, sizep

wave = np.linspace(0.3,2.55, sizep) #(sizep,)
fav_wave = np.repeat(wave[np.newaxis,:], total_spectra, axis=0) #This is n repeated totalspectra, sizep

n.shape, favn.shape, k.shape, favk.shape, wave.shape, fav_wave.shape

((225,), (12, 225), (225,), (12, 225), (225,), (12, 225))

In [4]:
s1,s2,s3,s4 = 0.01,0.02,0.03,0.04
olds = np.hstack((np.repeat(s1, pangl, axis=0), np.repeat(s2, pangl, axis=0), 
           np.repeat(s3, pangl, axis=0), np.repeat(s4, pangl, axis=0)))[:, np.newaxis]
print(olds.shape)
print(olds)
s = np.hstack((s1,s2,s3,s4))
s = np.repeat(s, pangl, axis=0)[:,np.newaxis]
print(s.shape)
print(s)

(12, 1)
[[0.01]
 [0.01]
 [0.01]
 [0.02]
 [0.02]
 [0.02]
 [0.03]
 [0.03]
 [0.03]
 [0.04]
 [0.04]
 [0.04]]
(12, 1)
[[0.01]
 [0.01]
 [0.01]
 [0.02]
 [0.02]
 [0.02]
 [0.03]
 [0.03]
 [0.03]
 [0.04]
 [0.04]
 [0.04]]


In [5]:
d1,d2,d3,d4 = 38,63,106,150
d = np.hstack((np.repeat(d1, pangl, axis=0), np.repeat(d2, pangl, axis=0), 
           np.repeat(d3, pangl, axis=0), np.repeat(d4, pangl, axis=0)))[:, np.newaxis]
d.shape 

(12, 1)

In [6]:
b1,b2,b3,b4 = 0.3, 0.4, 0.5, 0.6
b = np.hstack((np.repeat(b1,sizep,axis=0),np.repeat(b3,sizep,axis=0),
               np.repeat(b2,sizep,axis=0),np.repeat(b4,sizep,axis=0)))
c = -b
b.shape, c.shape

((900,), (900,))

In [14]:
b_ustk = np.split(b, gs)
c_ustk = np.split(c, gs)
blst = []
clst = []
for i in range(gs):
    blst.append(np.repeat(b_ustk[i][np.newaxis, :], pangl, axis=0))
    clst.append(np.repeat(c_ustk[i][np.newaxis, :], pangl, axis=0))
allb = np.vstack(blst) #nfiles, sizep
allc = np.vstack(clst) #nfiles, sizep
allb.shape, allc.shape

##Scale and offset

array([[0.3, 0.3, 0.3, ..., 0.3, 0.3, 0.3],
       [0.3, 0.3, 0.3, ..., 0.3, 0.3, 0.3],
       [0.3, 0.3, 0.3, ..., 0.3, 0.3, 0.3],
       ...,
       [0.6, 0.6, 0.6, ..., 0.6, 0.6, 0.6],
       [0.6, 0.6, 0.6, ..., 0.6, 0.6, 0.6],
       [0.6, 0.6, 0.6, ..., 0.6, 0.6, 0.6]])

## INIT_REFRACTION

In [14]:
def init_refraction(n):
    n = np.asarray(n)
    numer = (n - 1) ** 2
    denom = (n + 1) ** 2
    # approximate surface reflection coefficient S_E
    Se = numer / denom + 0.05
    # approximate internal scattering coefficient S_I
    Si = 1.014 - 4 / (n * denom)
    return Se, Si

In [15]:
Se, Si = init_refraction(favn) ## This works
# Se is 12,225 == same down different across because that is how n is
# ditto on Si

## SCATTERING EFFICIENCY

In [16]:
Alpha = (4 * np.pi * favk) / fav_wave
Alpha.shape

(12, 225)

In [17]:
s

array([[0.01],
       [0.01],
       [0.01],
       [0.02],
       [0.02],
       [0.02],
       [0.03],
       [0.03],
       [0.03],
       [0.04],
       [0.04],
       [0.04]])

In [18]:
Alpha_s = Alpha + s
Alpha_s

array([[0.42887902, 0.42254608, 0.4166106 , ..., 0.25658204, 0.25649037,
        0.25639942],
       [0.42887902, 0.42254608, 0.4166106 , ..., 0.25658204, 0.25649037,
        0.25639942],
       [0.42887902, 0.42254608, 0.4166106 , ..., 0.25658204, 0.25649037,
        0.25639942],
       ...,
       [0.45887902, 0.45254608, 0.4466106 , ..., 0.28658204, 0.28649037,
        0.28639942],
       [0.45887902, 0.45254608, 0.4466106 , ..., 0.28658204, 0.28649037,
        0.28639942],
       [0.45887902, 0.45254608, 0.4466106 , ..., 0.28658204, 0.28649037,
        0.28639942]])

In [19]:
Alpha.shape, Alpha_s.shape
tmp = np.sqrt(Alpha/(Alpha_s))

In [20]:
ri = (1-tmp) / (1+tmp) 

In [21]:
tmp = np.exp(-d * np.sqrt(Alpha*Alpha_s))

In [22]:
#Hapke1993 equation 11.15a
THETA = (ri + tmp) / (1 + ri*tmp)

In [23]:
#Hapke1993 equation 11.14 for w aka single scattering albedo
scat_eff = Se + (1-Se)*(((1-Si)*THETA)/(1 - Si*THETA))

In [24]:
scat_eff.shape, scat_eff

((12, 225),
 array([[0.12593267, 0.12632402, 0.12670908, ..., 0.18951563, 0.18982301,
         0.19013085],
        [0.12593267, 0.12632402, 0.12670908, ..., 0.18951563, 0.18982301,
         0.19013085],
        [0.12593267, 0.12632402, 0.12670908, ..., 0.18951563, 0.18982301,
         0.19013085],
        ...,
        [0.1630722 , 0.16401489, 0.16493305, ..., 0.26225747, 0.26265481,
         0.26305254],
        [0.1630722 , 0.16401489, 0.16493305, ..., 0.26225747, 0.26265481,
         0.26305254],
        [0.1630722 , 0.16401489, 0.16493305, ..., 0.26225747, 0.26265481,
         0.26305254]]))

## INIT ANGLES

In [25]:
ti1 = np.array([20,40,60])
te1 = np.array([0,0,0])
thetai = np.deg2rad(np.repeat(ti1[np.newaxis, :], gs, axis=0).flatten()[:, np.newaxis])
thetae = np.deg2rad(np.repeat(te1[np.newaxis, :], gs, axis=0).flatten()[:,np.newaxis])
thetai, thetae, thetai.shape, thetae.shape

(array([[0.34906585],
        [0.6981317 ],
        [1.04719755],
        [0.34906585],
        [0.6981317 ],
        [1.04719755],
        [0.34906585],
        [0.6981317 ],
        [1.04719755],
        [0.34906585],
        [0.6981317 ],
        [1.04719755]]), array([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]), (12, 1), (12, 1))

In [26]:
thetai = np.asarray(thetai)
thetae = np.asarray(thetae)
cosg = np.cos(np.abs(thetae - thetai))
u0 = np.cos(thetai)
u = np.cos(thetae)
cosg.shape, u0.shape, u.shape

((12, 1), (12, 1), (12, 1))

In [27]:
cosg

array([[0.93969262],
       [0.76604444],
       [0.5       ],
       [0.93969262],
       [0.76604444],
       [0.5       ],
       [0.93969262],
       [0.76604444],
       [0.5       ],
       [0.93969262],
       [0.76604444],
       [0.5       ]])

In [28]:
u0

array([[0.93969262],
       [0.76604444],
       [0.5       ],
       [0.93969262],
       [0.76604444],
       [0.5       ],
       [0.93969262],
       [0.76604444],
       [0.5       ],
       [0.93969262],
       [0.76604444],
       [0.5       ]])

In [29]:
u

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

## RADIANCE COEFF

In [30]:
b_ustk = np.split(b, gs)
blst = []
for i in range(gs):
    blst.append(np.repeat(b_ustk[i][np.newaxis, :], pangl, axis=0))
allb = np.vstack(blst)
allb.shape, allb

((12, 225), array([[0.3, 0.3, 0.3, ..., 0.3, 0.3, 0.3],
        [0.3, 0.3, 0.3, ..., 0.3, 0.3, 0.3],
        [0.3, 0.3, 0.3, ..., 0.3, 0.3, 0.3],
        ...,
        [0.6, 0.6, 0.6, ..., 0.6, 0.6, 0.6],
        [0.6, 0.6, 0.6, ..., 0.6, 0.6, 0.6],
        [0.6, 0.6, 0.6, ..., 0.6, 0.6, 0.6]]))

In [31]:
c_ustk = np.split(c, gs)
clst = []
for i in range(gs):
    clst.append(np.repeat(c_ustk[i][np.newaxis, :], pangl, axis=0))
allc = np.vstack(clst)
allc.shape, allc

((12, 225), array([[-0.3, -0.3, -0.3, ..., -0.3, -0.3, -0.3],
        [-0.3, -0.3, -0.3, ..., -0.3, -0.3, -0.3],
        [-0.3, -0.3, -0.3, ..., -0.3, -0.3, -0.3],
        ...,
        [-0.6, -0.6, -0.6, ..., -0.6, -0.6, -0.6],
        [-0.6, -0.6, -0.6, ..., -0.6, -0.6, -0.6],
        [-0.6, -0.6, -0.6, ..., -0.6, -0.6, -0.6]]))

In [32]:
#LegendrePhaseMixin
Pg1 = 1 + allb * cosg + allc * (1.5*(cosg**2)-0.5)
Pg1.shape, Pg1

((12, 225),
 array([[1.03454779, 1.03454779, 1.03454779, ..., 1.03454779, 1.03454779,
         1.03454779],
        [1.11574249, 1.11574249, 1.11574249, ..., 1.11574249, 1.11574249,
         1.11574249],
        [1.1875    , 1.1875    , 1.1875    , ..., 1.1875    , 1.1875    ,
         1.1875    ],
        ...,
        [1.06909557, 1.06909557, 1.06909557, ..., 1.06909557, 1.06909557,
         1.06909557],
        [1.23148499, 1.23148499, 1.23148499, ..., 1.23148499, 1.23148499,
         1.23148499],
        [1.375     , 1.375     , 1.375     , ..., 1.375     , 1.375     ,
         1.375     ]]))

In [33]:
# double Heyney-Greenstein phase function P(g)
x0 = allb * allb + 1
x1 = 2 * allb * cosg
x3 = (x0 - x1) ** 1.5
x4 = (x0 + x1) ** 1.5
Pg2 = (x0 - 2) * (x3*(allc-1) - x4*(allc+1)) / (2 * x3 * x4)
Pg2.shape, Pg2

((12, 225),
 array([[1.11256892, 1.11256892, 1.11256892, ..., 1.11256892, 1.11256892,
         1.11256892],
        [0.94300402, 0.94300402, 0.94300402, ..., 0.94300402, 0.94300402,
         0.94300402],
        [0.81453352, 0.81453352, 0.81453352, ..., 0.81453352, 0.81453352,
         0.81453352],
        ...,
        [1.27322272, 1.27322272, 1.27322272, ..., 1.27322272, 1.27322272,
         1.27322272],
        [0.58624048, 0.58624048, 0.58624048, ..., 0.58624048, 0.58624048,
         0.58624048],
        [0.37978112, 0.37978112, 0.37978112, ..., 0.37978112, 0.37978112,
         0.37978112]]))

In [34]:
def r0(scat_eff):
    gamma = np.sqrt(1 - scat_eff)
    return (1-gamma) / (1+gamma)
testr0 = r0(scat_eff)
testr0.shape, testr0

((12, 225),
 array([[0.03363677, 0.0337486 , 0.03385869, ..., 0.05248254, 0.05257711,
         0.05267186],
        [0.03363677, 0.0337486 , 0.03385869, ..., 0.05248254, 0.05257711,
         0.05267186],
        [0.03363677, 0.0337486 , 0.03385869, ..., 0.05248254, 0.05257711,
         0.05267186],
        ...,
        [0.04447501, 0.0447562 , 0.04503037, ..., 0.07589388, 0.07602779,
         0.07616189],
        [0.04447501, 0.0447562 , 0.04503037, ..., 0.07589388, 0.07602779,
         0.07616189],
        [0.04447501, 0.0447562 , 0.04503037, ..., 0.07589388, 0.07602779,
         0.07616189]]))

In [35]:
def Hu(scat_eff, u):
    tr0 = r0(scat_eff)
    tmp_gamma = np.sqrt(1 - scat_eff)
    return 1/(1-(1-tmp_gamma)*u*(tr0 + (1 - 0.5*tr0 - u*tr0)*np.log((1 + u)/u)))

testHu = Hu(scat_eff, u)
testHu.shape, testHu

((12, 225),
 array([[1.04714899, 1.04730748, 1.0474635 , ..., 1.07402193, 1.07415762,
         1.07429357],
        [1.04714899, 1.04730748, 1.0474635 , ..., 1.07402193, 1.07415762,
         1.07429357],
        [1.04714899, 1.04730748, 1.0474635 , ..., 1.07402193, 1.07415762,
         1.07429357],
        ...,
        [1.06256335, 1.06296472, 1.06335614, ..., 1.10786669, 1.10806176,
         1.10825713],
        [1.06256335, 1.06296472, 1.06335614, ..., 1.10786669, 1.10806176,
         1.10825713],
        [1.06256335, 1.06296472, 1.06335614, ..., 1.10786669, 1.10806176,
         1.10825713]]))

In [36]:
def Hu_Hu0(scat_eff, u, u0):
    tHu = Hu(scat_eff, u=u)
    tHu0 = Hu(scat_eff, u=u0)
    return tHu, tHu0

tHu, tHu0 = Hu_Hu0(scat_eff, u, u0)
tHu.shape, tHu, tHu0.shape, tHu0

((12, 225),
 array([[1.04714899, 1.04730748, 1.0474635 , ..., 1.07402193, 1.07415762,
         1.07429357],
        [1.04714899, 1.04730748, 1.0474635 , ..., 1.07402193, 1.07415762,
         1.07429357],
        [1.04714899, 1.04730748, 1.0474635 , ..., 1.07402193, 1.07415762,
         1.07429357],
        ...,
        [1.06256335, 1.06296472, 1.06335614, ..., 1.10786669, 1.10806176,
         1.10825713],
        [1.06256335, 1.06296472, 1.06335614, ..., 1.10786669, 1.10806176,
         1.10825713],
        [1.06256335, 1.06296472, 1.06335614, ..., 1.10786669, 1.10806176,
         1.10825713]]),
 (12, 225),
 array([[1.04628208, 1.04643752, 1.04659053, ..., 1.07262287, 1.0727558 ,
         1.07288898],
        [1.04334846, 1.04349359, 1.04363645, ..., 1.06789897, 1.06802264,
         1.06814655],
        [1.03696067, 1.03708357, 1.03720453, ..., 1.05766914, 1.05777305,
         1.05787715],
        ...,
        [1.06139459, 1.06178797, 1.0621716 , ..., 1.10575787, 1.10594871,
         1

## Lambertian Mixing

In [37]:
tHu, tHu0 = Hu_Hu0(scat_eff, u, u0)

In [38]:
#Lambertian Mixing
Bg1 = 1
tmp = Pg1 * Bg1 + tHu*tHu0 - 1

In [194]:
#Hapke1993 equation 10.4
rc11 = tmp * (scat_eff/4.) / (u + u0)

In [195]:
rc11.shape, rc11

((12, 225),
 array([[0.01834365, 0.01840601, 0.0184674 , ..., 0.0289831 , 0.02903716,
         0.02909133],
        [0.02154   , 0.02161261, 0.0216841 , ..., 0.03387505, 0.03393746,
         0.03399998],
        [0.02672611, 0.02681533, 0.02690316, ..., 0.04180272, 0.04187859,
         0.04195461],
        ...,
        [0.02515608, 0.02531934, 0.02547859, ..., 0.04374336, 0.0438241 ,
         0.04390497],
        [0.03128134, 0.03148108, 0.03167585, ..., 0.05378114, 0.05387788,
         0.05397476],
        [0.04048244, 0.04073695, 0.04098509, ..., 0.06885343, 0.06897413,
         0.06909501]]))

In [213]:
#Lambertian Mixing with HG thingy
Bg1 = 1
tmp = Pg2 * Bg1 + tHu*tHu0 - 1

In [214]:
#Hapke1993 equation 10.4
rc12 = tmp * (scat_eff/4.) / (u + u0)

In [215]:
rc12.shape, rc12

((12, 225),
 array([[0.01958072, 0.01964682, 0.01971188, ..., 0.03081595, 0.03087283,
         0.03092983],
        [0.01842784, 0.01849066, 0.01855251, ..., 0.02915954, 0.02921413,
         0.02926883],
        [0.01885896, 0.01892359, 0.01898723, ..., 0.02992587, 0.02998228,
         0.0300388 ],
        ...,
        [0.02939452, 0.02958191, 0.02976465, ..., 0.05048651, 0.05057712,
         0.05066787],
        [0.01632835, 0.01644122, 0.01655139, ..., 0.02965224, 0.02971204,
         0.02977196],
        [0.01336489, 0.01346215, 0.01355712, ..., 0.02514788, 0.02520193,
         0.0252561 ]]))

## Isotropic Mixing

In [211]:
isow = np.linspace(0.01,1, sizep)
print(isow.shape, isow)
print("-"*120)
isoHu, isoHu0 = Hu_Hu0(isow, u, u0)
rc_denom = isow * isoHu0 * isoHu
print(rc_denom.shape, rc_denom)

(225,) [0.01       0.01441964 0.01883929 0.02325893 0.02767857 0.03209821
 0.03651786 0.0409375  0.04535714 0.04977679 0.05419643 0.05861607
 0.06303571 0.06745536 0.071875   0.07629464 0.08071429 0.08513393
 0.08955357 0.09397321 0.09839286 0.1028125  0.10723214 0.11165179
 0.11607143 0.12049107 0.12491071 0.12933036 0.13375    0.13816964
 0.14258929 0.14700893 0.15142857 0.15584821 0.16026786 0.1646875
 0.16910714 0.17352679 0.17794643 0.18236607 0.18678571 0.19120536
 0.195625   0.20004464 0.20446429 0.20888393 0.21330357 0.21772321
 0.22214286 0.2265625  0.23098214 0.23540179 0.23982143 0.24424107
 0.24866071 0.25308036 0.2575     0.26191964 0.26633929 0.27075893
 0.27517857 0.27959821 0.28401786 0.2884375  0.29285714 0.29727679
 0.30169643 0.30611607 0.31053571 0.31495536 0.319375   0.32379464
 0.32821429 0.33263393 0.33705357 0.34147321 0.34589286 0.3503125
 0.35473214 0.35915179 0.36357143 0.36799107 0.37241071 0.37683036
 0.38125    0.38566964 0.39008929 0.39450893 0.39892857 0

In [212]:
ff = 0.03
tmp = -1.209 * ff**(2./3)
PoreK = np.log1p(tmp) / tmp

# perform the change of variables to account for porosity outside of H(u)
# and H(u0) equations for simplicity, i.e. H(u) will become H(u/PoreK)
# see Hapke 2012b (the book) for details.
uK = u / PoreK
u0K = u0 / PoreK
tHu, tHu0 = Hu_Hu0(scat_eff, uK, u0K)

tmp = Pg1 * 1 + tHu*tHu0 - 1
numer = PoreK * tmp * scat_eff
rc21 = numer / rc_denom
print(rc21.shape, rc21)

(12, 225) [[15.00591389 10.40991811  7.96974094 ...  0.03999287  0.03677575
   0.02984888]
 [16.04774146 11.13337365  8.52416969 ...  0.04665402  0.04318477
   0.03568942]
 [16.92067137 11.74098493  8.99092906 ...  0.05817983  0.05443297
   0.04627691]
 ...
 [20.56919662 14.3131261  10.99018191 ...  0.06029559  0.05544408
   0.04499995]
 [23.29752342 16.21146233 12.44770251 ...  0.07400721  0.06850085
   0.05660893]
 [25.62386866 17.83213303 13.69361031 ...  0.09576372  0.08959076
   0.07616192]]


In [216]:
# Isotropic with HG thingy

ff = 0.03
tmp = -1.209 * ff**(2./3)
PoreK = np.log1p(tmp) / tmp

# perform the change of variables to account for porosity outside of H(u)
# and H(u0) equations for simplicity, i.e. H(u) will become H(u/PoreK)
# see Hapke 2012b (the book) for details.
uK = u / PoreK
u0K = u0 / PoreK
tHu, tHu0 = Hu_Hu0(scat_eff, uK, u0K)

tmp = Pg2 * 1 + tHu*tHu0 - 1
numer = PoreK * tmp * scat_eff
rc22 = numer / rc_denom
print(rc22.shape, rc22)

(12, 225) [[16.04351013 11.1295151   8.52050332 ...  0.04262918  0.03919941
   0.03181556]
 [13.7500315   9.53971147  7.3043054  ...  0.04025629  0.03726406
   0.03079739]
 [11.95733579  8.2977806   6.35480048 ...  0.04174619  0.03906058
   0.03321034]
 ...
 [24.08446451 16.75754265 12.86583608 ...  0.0698404   0.06421805
   0.05211885]
 [12.18348389  8.48236398  6.51644597 ...  0.04093649  0.03789917
   0.0313268 ]
 [ 8.47387574  5.90299843  4.53740851 ...  0.03508096  0.0328328
   0.02792267]]
